In [2]:
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
df = pd.read_excel('Online Retail.xlsx')
display(df.head())

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [11]:
df.isna().sum()

,0
InvoiceNo,0
StockCode,0
Description,1454
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,135080
Country,0


In [12]:
print("jumlah duplikasi data :", df.duplicated().sum())

jumlah duplikasi data : 5268


In [13]:
df.describe()

,Quantity,InvoiceDate,UnitPrice,CustomerID
count,541909.000000,541909,541909.000000,406829.000000
mean,9.552250,2011-07-04 13:34:57.156386048,4.611114,15287.690570
min,-80995.000000,2010-12-01 08:26:00,-11062.060000,12346.000000
25%,1.000000,2011-03-28 11:34:00,1.250000,13953.000000
50%,3.000000,2011-07-19 17:17:00,2.080000,15152.000000
75%,10.000000,2011-10-19 11:27:00,4.130000,16791.000000
max,80995.000000,2011-12-09 12:50:00,38970.000000,18287.000000
std,218.081158,NaN,96.759853,1713.600303


In [14]:
# MEnghilangkan duplikasi data
df.drop_duplicates(inplace=True)

In [15]:
print("jumlah duplikasi data :", df.duplicated().sum())

jumlah duplikasi data : 0


In [16]:
# Mengatasi missing value di deskripsi
# Buat mapping StockCode -> Description (ambil deskripsi pertama yang tidak kosong)
mapping = df.dropna(subset=['Description']).drop_duplicates('StockCode')\
           .set_index('StockCode')['Description'].to_dict()

In [17]:
# Isi missing value Description berdasarkan mapping
df['Description'] = df.apply(
    lambda row: mapping.get(row['StockCode'], "Unknown") if pd.isna(row['Description']) else row['Description'],
    axis=1
)

In [18]:
df.isna().sum()

,0
InvoiceNo,0
StockCode,0
Description,0
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,135037
Country,0


In [24]:
#Hapus baris CustomerID yang kosong, karena untuk analisis pelanggan akan sulit
#Ini dirubah dulu karena kemungkinan besar, data kosongnya itu string atau spasi kosong sehingga saat langsung drop subset (drop baris yang kosong, ga ke efek)
# Ubah string kosong atau spasi jadi NaN
df['CustomerID'] = df['CustomerID'].replace(r'^\s*$', np.nan, regex=True)

In [25]:
# Sekarang drop NaN
df_clean = df.dropna(subset=['CustomerID'])

print("Jumlah baris awal:", len(df))
print("Jumlah baris setelah drop:", len(df_clean))

Jumlah baris awal: 536641
Jumlah baris setelah drop: 401604


In [27]:

print(df_clean['CustomerID'].isna().sum())  # cek data yang sudah bersih, disimpan dalam df baru (df_clean)


0


In [39]:
#Buat Kolom Baru TotalPrice untuk mendapatkan hasil keuntungan
df_clean['TotalPrice'] = df_clean['UnitPrice'] * df_clean['Quantity']


/tmp/ipython-input-4120609710.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['TotalPrice'] = df_clean['UnitPrice'] * df_clean['Quantity']


In [40]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 401604 entries, 0 to 541908
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    401604 non-null  object        
 1   StockCode    401604 non-null  object        
 2   Description  401604 non-null  object        
 3   Quantity     401604 non-null  int64         
 4   InvoiceDate  401604 non-null  datetime64[ns]
 5   UnitPrice    401604 non-null  float64       
 6   CustomerID   401604 non-null  float64       
 7   Country      401604 non-null  object        
 8   IsCancelled  401604 non-null  bool          
 9   TotalPrice   401604 non-null  float64       
dtypes: bool(1), datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 31.0+ MB


In [37]:
#Kita buat klasisifikasi kolom InvoiceNo, karena ada yang awalannya C berarti di cancel, dan ada yang gak, artinya transaksi berhasil
df_clean['IsCancelled'] = df_clean['InvoiceNo'].astype(str).str.startswith('C')

/tmp/ipython-input-2076347040.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['IsCancelled'] = df_clean['InvoiceNo'].astype(str).str.startswith('C')


In [43]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 401604 entries, 0 to 541908
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    401604 non-null  object        
 1   StockCode    401604 non-null  object        
 2   Description  401604 non-null  object        
 3   Quantity     401604 non-null  int64         
 4   InvoiceDate  401604 non-null  datetime64[ns]
 5   UnitPrice    401604 non-null  float64       
 6   CustomerID   401604 non-null  float64       
 7   Country      401604 non-null  object        
 8   IsCancelled  401604 non-null  bool          
 9   TotalPrice   401604 non-null  float64       
 10  Year         401604 non-null  int32         
 11  Month        401604 non-null  int32         
 12  Day          401604 non-null  int32         
dtypes: bool(1), datetime64[ns](1), float64(3), int32(3), int64(1), object(4)
memory usage: 35.6+ MB


In [42]:
#Kita pisah dulu InvoiceDate biar bisa analisis dengan mudah
# Pastikan kolom InvoiceDate sudah datetime
df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'])

# Tambah kolom baru Year, Month, Day
df_clean['Year'] = df_clean['InvoiceDate'].dt.year
df_clean['Month'] = df_clean['InvoiceDate'].dt.month
df_clean['Day'] = df_clean['InvoiceDate'].dt.day


/tmp/ipython-input-2974502820.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'])
/tmp/ipython-input-2974502820.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['Year'] = df_clean['InvoiceDate'].dt.year
/tmp/ipython-input-2974502820.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

In [44]:
#Export ke CSV
df_clean.to_csv("cleaned_retail.csv", index=False)
